In [70]:
import tabula
import pandas

In [71]:
filepath = "./Rec_Task/d9f8e6d9-660b-4505-86f9-952e45ca6da0.pdf"

In [72]:
tables =  tabula.read_pdf(filepath, pages="1", area=(338.21617916107175, 55.43917568206787, 444.62963047027586, 527.2302814865112))

In [73]:
def is_not_nan(val):
    return val == val

def concat_rows(tbl, common_col):
    tmp_df = pandas.DataFrame(columns=tbl.columns)
    ind = -1
    for index, row in tbl.iterrows():
        if(is_not_nan(row[common_col])):
            ind+=1
            tmp_df.loc[ind]=['']*len(tbl.columns)
        for col in tbl.columns:
            tmp_df.loc[ind][col]+=" "+str(row[col] if is_not_nan(row[col]) else "")
            tmp_df.loc[ind][col]=tmp_df.loc[ind][col].strip()
    return tmp_df

def merge_col_row(tbl, last_row):
    col = [str(i if is_not_nan(i) and i.find("Unnamed:") == -1 else "") for i in tbl.columns.to_list()]
    for row_ind in range(last_row+1):
        for col_ind in range(len(col)):
            row_item = tbl.loc[row_ind][col_ind]
            col[col_ind] += " " if len(col[col_ind]) > 0 and col[col_ind][-1]!="/" and col[col_ind][-1]!="(" else ""
            col[col_ind] += str(row_item) if is_not_nan(row_item) else ""
            col[col_ind] = col[col_ind].strip()
    
    tbl.columns = col
    tbl = tbl.iloc[last_row+1:].reset_index(drop=True)
    return tbl

def fill_blank_cell(tbl, column_name):
    for index, row in tbl.iterrows():
        if tbl.loc[index][column_name] == "":
            if index != 0:
                tbl.loc[index][column_name] = tbl.loc[index-1][column_name]
            else:
                tbl.loc[index][column_name] = "-"
    return tbl

In [74]:
print(len(tables))

1


In [75]:
tables

[            Date          Name of the analyst/investor        Type Location
 0  April 4, 2018        Motilal Oswal Asset Management  One-on-One   Mumbai
 1            NaN                       Company Limited     meeting      NaN
 2            NaN                         Credit Suisse  Voice call        -
 3            NaN                                     I         NaN      NaN
 4  April 5, 2018  Maybank Eng Securities India Private  One-on-One   Mumbai
 5            NaN                               Limited     meeting      NaN]

In [76]:
t1 = tables[0]
t1

,Date,Name of the analyst/investor,Type,Location
0,"April 4, 2018",Motilal Oswal Asset Management,One-on-One,Mumbai
1,NaN,Company Limited,meeting,NaN
2,NaN,Credit Suisse,Voice call,-
3,NaN,I,NaN,NaN
4,"April 5, 2018",Maybank Eng Securities India Private,One-on-One,Mumbai
5,NaN,Limited,meeting,NaN


In [77]:
# merge rows based on a particular column
t1 = concat_rows(t1, 'Location')

In [81]:
# populating empty cells from above cells
t1 = fill_blank_cell(t1, 'Date')

# manually cleaning data 
t1.loc[1][1] = t1.loc[1][1][0:-2]

In [82]:
t1

,Date,Name of the analyst/investor,Type,Location
0,"April 4, 2018",Motilal Oswal Asset Management Company Limited,One-on-One meeting,Mumbai
1,"April 4, 2018",Credit Su,Voice call,-
2,"April 5, 2018",Maybank Eng Securities India Private Limited,One-on-One meeting,Mumbai
